In [ ]:
# cil imports
from cil.framework import ImageData, ImageGeometry
from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.plugins.astra import ProjectionOperator

from cil.processors import CentreOfRotationCorrector, Slicer, \
    Binner, Masker, MaskGenerator, TransmissionAbsorptionConverter

from cil.plugins.astra import FBP

from cil.utilities import dataexample
from cil.utilities.display import show2D, show_geometry

from cil.optimisation.functions import LeastSquares, IndicatorBox
from cil.optimisation.algorithms import FISTA


# External imports
import numpy as np
import matplotlib.pyplot as plt
import logging

In [ ]:
# Set up a reader object pointing to the Nexus data set
data_raw = dataexample.SYNCHROTRON_PARALLEL_BEAM_DATA.get()

In [ ]:
#preprocessing
data_raw /= data_raw.get_slice(vertical=20).mean()
data_exp = TransmissionAbsorptionConverter()(data_raw)
data_crop = Slicer(roi={'vertical': (1, None)})(data_exp)
data_centred = CentreOfRotationCorrector.xcorrelation()(data_crop)


In [ ]:
data_centred.reorder('astra')
ag = data_centred.geometry
ig = ag.get_ImageGeometry()
A = ProjectionOperator(ig, ag, device="gpu")

In [ ]:
b = data_centred
F = LeastSquares(A, b)
G = IndicatorBox(lower=0.0)
x0 = ig.allocate(0.0)


myFISTANN = FISTA(f=F, 
                  g=G, 
                  initial=x0, 
                  update_objective_interval = 10)

In [ ]:
from cil.optimisation.utilities import callbacks
from custom_callbacks import ShowIterates, SaveSlices
import os

cb1 = callbacks.ProgressCallback() # This is the progress bar 
cb2 = callbacks.TextProgressCallback()
cb3 = callbacks.LogfileCallback('log.txt')
interval = 10 # the following 2 callbacks take parameter iterations - so they will happen every 10 iterations
cb4 = SaveSlices(interval=interval, file_name="fista", dir_path="./callback/") 
cb5 = ShowIterates(interval) # can add fix_range parameter here

In [ ]:
myFISTANN.run(30, callbacks= [cb1, cb2, cb3, cb4, cb5])